
# CS6220 Homework 2
# Map Reduce: Friends of Friends

Include your code in this file. Make sure the below piece of code is at the top, as we will use that variable for testing.

### Tips and tricks
* Besides the Spark documentation, use the REPL feature heavily, since you'll be able to see functionality and functions.
* One function you may find useful is the `collect()` function that can collects the RDD from all machines and brings it into memory. This is only feasible for small datasets, and it will allow you to effectively debug.
* You can mount the `datapath` from a Google Drive. That way you won't have to keep uploading to Google Colab.
  * Try using the following code block:
  
  ```
     from google.colab import drive
     drive.mount('/content/drive')
  ```

* The total runtime is around 10 minutes, where you'll only notice in the reduce step. Spark is a lazy evaluator, and only when there's a `collect` or other evaluator step will you notice the lag.

In [1]:
#@title Data path for file. We will use the variable `data_path` for grading.
datapath="/content/drive/MyDrive/path-to-soc-LiveJournal1Adj.txt" #@param 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Your Code Below

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=713d8e2ecf07294224ee7cd41871603d1cf17f7bfef3c651522342105073e1bb
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import itertools 

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Read data file and process data

In [13]:
lines = sc.textFile(datapath, 1).map(lambda x: x.split("\t"))


In [14]:
# turn data type into int(for sorting after)
# split friends into a list
def split_to_int(split):
  
  user = int(split[0])
  if len(split[1]) == 0:
    friends = []
  else:
    friends = list(map(lambda x: int(x), split[1].split(",")))

  return (user, friends)


def make_pair(friend_list):
  user = friend_list[0]
  friends = friend_list[1]

  already_friends = [((user, friend), 0) for friend in friends]
  have_mutual_friend = [(have_mutual, 1)for have_mutual in itertools.combinations(friends, 2)]

  return already_friends+have_mutual_friend


# [user:int, friends:list[int]]
user_friend_pair = lines.map(split_to_int).flatMap(make_pair)
user_friend_pair.cache()

mutual_friend_counts = user_friend_pair.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .map(lambda edge: (edge[0], sum(edge[1])))


mutual_friend_counts.collect()

Py4JJavaError: ignored